In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px 

import warnings

warnings.simplefilter(action='ignore')

In [ ]:
df = pd.read_csv(r'../input/travel-insurance-prediction-data/TravelInsurancePrediction.csv')
df.shape

In [ ]:
data = df.copy()

In [ ]:
data.isna().sum()

In [ ]:
data.info()

In [ ]:
data.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
data.head()

## EDA

In [ ]:
def pie_chart(cols):
    """ 
    A funtion for plotting a pir chart of all the distinct
    value present in the dataset wrt to a certain column
    
    """
    for i in cols:
        values = data[i].value_counts()
        labels = data[i].value_counts().index    
        
        fig = px.pie(data, names=labels, values=values, hole=.3, color_discrete_sequence = ['pink', 'red'])
        fig.update_layout(title=f'Distribution based on {i}', template='plotly_dark', hoverlabel=dict(
            font_size=18,
            font_family='Arial',
        ))
        fig.update_traces(hovertemplate='Count of %{label}: %{value}')
        fig.show()

In [ ]:
pie_list = [i for i in data.columns[:-1] if len(data[i].value_counts()) < 20] # list of columns having less than 20 unique values
pie_list  

In [ ]:
pie_chart(pie_list)

In [ ]:
data.head()

Checking for outliers in the Annual Income column

In [ ]:
plt.figure(figsize=(8, 6))
sns.distplot(data['AnnualIncome'])
plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder, Normalizer, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

from imblearn.over_sampling import RandomOverSampler

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
data['TravelInsurance'].value_counts()

In [ ]:
obj_cols = [i for i in data.columns if data[i].dtype == 'O']
obj_cols

In [ ]:
label_enc = LabelEncoder()

In [ ]:
for i in obj_cols:
    data[i] = label_enc.fit_transform(data[i])

In [ ]:
data.head()

In [ ]:
X = data.iloc[:, :-1]
y = data['TravelInsurance']

In [ ]:
ros = RandomOverSampler(random_state=42)

In [ ]:
x_ros, y_ros = ros.fit_resample(X, y)
y_ros.value_counts()

In [ ]:
x_ros.head()

In [ ]:
x_ros.shape

In [ ]:
def normalizer(x_train, x_test):
    scaler = Normalizer()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    return x_train, x_test

In [ ]:
def standardscaler(x_train, x_test):
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    
    return x_train, x_test

In [ ]:
def minmax(x_train, x_test):
    scaler = MinMaxScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    return x_train, x_test

In [ ]:
def best_model(X, y, scaler, algo):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True)
    
    X_train = np.asarray(X_train)
    X_test = np.asarray(X_test)
        
    X_train, X_test = scaler(X_train, X_test)
    
    y_train = np.asarray(y_train)
    y_test = np.asarray(y_test)
    
    model = algo()
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    
    training_score = model.score(X_train, y_train)
    acc = accuracy_score(y_test, predictions)
    con = confusion_matrix(y_test, predictions)
    report = classification_report(y_test, predictions)
    print(f'Training Score: {training_score}')
    print(f'Accuracy Score: {acc}')
    print(f'Confusion Matrix: {con}')
    print(f'Classification Report: {report}')
    return acc, training_score

In [ ]:
list_algo = [LogisticRegression, KNeighborsClassifier, DecisionTreeClassifier, RandomForestClassifier, XGBClassifier]

list_of_accuracy = []
list_of_score = []

for i in list_algo:
    accuracyscore, trainingscore = best_model(x_ros, y_ros, normalizer, i)
    list_of_accuracy.append(accuracyscore)
    list_of_score.append(trainingscore) 

In [ ]:
list_of_accuracy1 = []
list_of_score1 = []
for i in list_algo:
    accuracyscore, trainingscore = best_model(x_ros, y_ros, standardscaler, i)
    list_of_accuracy1.append(accuracyscore)
    list_of_score1.append(trainingscore)    

In [ ]:
list_of_accuracy2 = []
list_of_score2 = []
for i in list_algo:
    accuracyscore, trainingscore = best_model(x_ros, y_ros, minmax, i)
    list_of_accuracy2.append(accuracyscore)
    list_of_score2.append(trainingscore)  

In [ ]:
acc_df = pd.DataFrame()
acc_df['Algo'] = ['LogisticRegression', 'KNeighborsClassifier', 'DecisionTreeClassifier', 'RandomForestClassifier', 'XGBClassifier']
acc_df['normazlier_acc'] = list_of_accuracy
acc_df['normazlier_training'] = list_of_score
acc_df['standard_acc'] = list_of_accuracy1
acc_df['standard_training'] = list_of_score1
acc_df['minmax_acc'] = list_of_accuracy2
acc_df['minmax_training'] = list_of_score2
acc_df

In [ ]:
fig = px.line(acc_df, x='Algo', y = ['normazlier_acc', 'standard_acc', 'minmax_acc'])
fig.update_layout(template='plotly_dark')
fig.show()